<a href="https://colab.research.google.com/github/alessiomongoli/Sentiment_Lexicon/blob/main/FileGenerale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import import_ipynb
import sys
sys.path.append('/content/drive/My Drive/Github/Colab Notebooks/project')

In [ ]:
%cd /content/drive/My Drive/Github/Colab Notebooks/project

/content/drive/My Drive/Github/Colab Notebooks/project


In [ ]:
import pandas as pd
import numpy as np


from nltk.corpus import stopwords
import nltk 
import string


from Data_induction.PreProcessing import preprocessing
from Data_induction.TrainSVM import LinearCoefficentsSVM
from glove.Glove import load_filter_glove_word
from Data_induction.Seed_Data import SeedDataset
from Neural_model.EarlyStopping import EarlyStopping
from Neural_model.Neural import RegressionModel
from Neural_model.Train_predict_NN import train, predict
from Utils.DataframeCreation import creation_dataframe
from Test.Experiment1 import experiment1


importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Data_induction/PreProcessing.ipynb


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Data_induction/TrainSVM.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/glove/Glove.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Data_induction/Seed_Data.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Neural_model/EarlyStopping.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Neural_model/Neural.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Neural_model/Train_predict_NN.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Utils/DataframeCreation.ipynb
importing Jupyter notebook from /content/drive/My Drive/Github/Colab Notebooks/project/Test/Experiment1.ipynb


In [ ]:
CATEGORY="reviews_Grocery_and_Gourmet_Food" #"reviews_Apps_for_Android" #reviews_Apps_for_Android.json.gz or reviews_Grocery_and_Gourmet_Food.json.gz or reviews_Movies_and_TV.json.gz
SAVE_DATAFRAME_FILE= False
CHECKPOINT_PATH = "/content/drive/MyDrive/Github/Colab Notebooks/project/checkpoint/checkpoint.pt"
NEGATION_TYPE='all_words' # 'normal' or 'all_words' or 'no_negation'
NEGATION_MEAN=True
REPOSITORY = '/content/drive/MyDrive/Github/Colab Notebooks/project/'

In [ ]:
"""" Preprocessing function:
1 Apre il file dove ci sono Recensioni e score.
2 Filtra le recensioni togliendo quelle con 3 stelle
3Usa la funzione find negation per aggiungere dei tag nelle recensioni dove trova vocaboli negativi {not never nor}
4)Crea due liste:recensioni, label (1 se >3 stelle, -1< 3 stelle)
5) Genera il document term Matrix 
6) Ritorna:
X Bag of word
Y array di label 1 o 0
Frequenze lista con le occurences
Vocabulary 
"""

X, y, frequencies, vocabulary = preprocessing(REPOSITORY+CATEGORY+".json.gz", NEGATION_TYPE, CATEGORY)

KeyboardInterrupt: ignored

In [ ]:
"""
Fa il training di SVM con le BOW e le label ed estrae i pesi che vengono considerate come polarità
"""
coefficents = LinearCoefficentsSVM(X, y, CATEGORY, NEGATION_TYPE)

In [ ]:
Data=creation_dataframe(vocabulary,coefficents, frequencies)
if SAVE_DATAFRAME_FILE==True:
  Data.to_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Data/'+CATEGORY+'_'+NEGATION_TYPE+'.csv',index=False)

In [ ]:
from IPython.display import Audio

sound_hey = "http://thecyberbuddy.com/sounds/beepbeepbeep.wav"

#Audio(sound_hey, autoplay=True) 

In [ ]:
#Audio(sound_hey, autoplay=True) 

#Cella per vedere quante negated ci sono 
Poi si discute se fare una funzione per le statistiche

In [ ]:
#c=[]
#for key, value in vocabulary.items():
#  if key.startswith('neg_'):
#    c.append(key)
#print(len(c))

In [ ]:
Data = pd.read_csv(REPOSITORY+'Data/'+CATEGORY+'_'+NEGATION_TYPE+'.csv')
Data

In [ ]:
Data = Data[Data['Frequence']>=500]
Data.reset_index(drop=True, inplace=True)

In [ ]:
Data

In [ ]:
if NEGATION_MEAN:
  negated={}
  prefix=len('neg_')
  for i in range(len(Data)):
    if str(Data['Token'][i]).startswith('neg_')==True:
      negated[Data['Token'][i][prefix:]]=Data['Polarity'][i]
  for i in range(len(Data)):
    if Data['Token'][i]in negated.keys():
      Data['Polarity'][i]=(negated[Data['Token'][i]]+Data['Polarity'][i]/2)

In [ ]:
Data

In [ ]:
%%time
#glove_embeddings_filtered= parole comuni tra glove e nostri tokens quindi automatimaticamente cancella le negation
glove_embeddings_filtered, glove_wo_pol= load_filter_glove_word(REPOSITORY+'glove/glove.840B.300d.pkl',list(Data['Token']), CATEGORY, NEGATION_TYPE)

In [ ]:
len(glove_embeddings_filtered)

In [ ]:
Data=Data[Data['Token'].isin(glove_embeddings_filtered.keys())]

In [ ]:
Data.reset_index(drop=True, inplace=True)
Data

In [ ]:
Data['Embedding']=Data['Token'].apply(lambda x: glove_embeddings_filtered[x])

In [ ]:
Data.reset_index(drop=True, inplace=True)

In [ ]:
Data

In [ ]:
#Data.to_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Data/provadata.csv',index=False)

In [ ]:
#Data = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Data/provadata.csv')

In [ ]:
Dataset=SeedDataset(list(Data['Token']),glove_embeddings_filtered, pol=list(Data['Polarity']))

In [ ]:
trained_model = train(Dataset, CATEGORY, NEGATION_TYPE)

In [ ]:
complete_result= Dataset.get_result()
non_seed_data={w:0 for w in glove_wo_pol.keys()}
# non_seed_dataset = SeedDataset(non_seed_data, list(glove_wo_pol), split='test')
non_seed_dataset = SeedDataset(list(non_seed_data.keys()),glove_wo_pol,split='test')

In [ ]:
results = predict(trained_model, non_seed_dataset, CATEGORY, NEGATION_TYPE)

In [ ]:
complete_result.update(results)

In [ ]:
if NEGATION_MEAN:
  df_IMDB = pd.read_csv(REPOSITORY+'IMDB-Dataset_preprocessed.csv')
  accuracy=experiment1(df_IMDB, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'IBDM')
else:
  df_IMDB = pd.read_csv(REPOSITORY+'IMDB-Dataset_preprocessed.csv')
  accuracy_IMDB=experiment1(df_IMDB, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'IBDM')

In [ ]:
accuracy_IMDB

In [ ]:
if NEGATION_MEAN:
  df_PlayStore = pd.read_csv(REPOSITORY+'googleplaystore_user_reviews_preprocessed.csv')
  accuracy_play=experiment1(df_PlayStore, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'GooglePlayStore')
else:
  df_PlayStore = pd.read_csv(REPOSITORY+'googleplaystore_user_reviews_preprocessed.csv')
  accuracy_play=experiment1(df_PlayStore, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'GooglePlayStore')
accuracy_play

In [ ]:
if NEGATION_MEAN:
  df_tripadvisor = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/tripadvisor_hotel_reviews_preprocessed.csv')
  accuracy_trip=experiment1(df_tripadvisor, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'TripAdvisor')
else:
  df_tripadvisor = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/tripadvisor_hotel_reviews_preprocessed.csv')
  accuracy_trip=experiment1(df_tripadvisor, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'TripAdvisor')
accuracy_trip

In [ ]:
if NEGATION_MEAN:
  df_tweets = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Tweets_preprocessed.csv')
  accuracy_tweet=experiment1(df_tweets, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'Tweets')
else:
  df_tweets = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Tweets_preprocessed.csv')
  accuracy_twet=experiment1(df_tweets, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'Tweets')
# valore prima 0.7131097825145135
accuracy_tweet

In [ ]:
if NEGATION_MEAN:
  df_tweets_4600 = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Tweets_preprocessed_4600.csv')
  accuracy_tweet46=experiment1(df_tweets_4600, complete_result, CATEGORY+'_'+NEGATION_TYPE+'_with_neg', 'Tweets4600')
else:
  df_tweets_4600 = pd.read_csv('/content/drive/MyDrive/Github/Colab Notebooks/project/Tweets_preprocessed_4600.csv')
  accuracy_tweet46=experiment1(df_tweets_4600, complete_result, CATEGORY+'_'+NEGATION_TYPE, 'Tweets4600')
accuracy_tweet46=

In [ ]:
from IPython.display import Audio

sound_hey = "http://thecyberbuddy.com/sounds/beepbeepbeep.wav"

#Audio(sound_hey, autoplay=True) 

In [ ]:
Audio(sound_hey, autoplay=True) 

In [ ]:
Audio(sound_hey, autoplay=True) 